<a href="https://colab.research.google.com/github/joshbarrass/stable-diffusion-colab-launcher/blob/master/Stable_Diffusion_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download the web UI...
%cd /content/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd /content/stable-diffusion-webui
# ...checkout current latest commit at time of writing (2023/02/28) to keep repo stable...
!git checkout 0cc0ee1

!pip install huggingface_hub
from huggingface_hub import hf_hub_url as hf_url

# helper classes for downloads
# download helper classes can implement any __init__ interface to accept
# any necessary args, but must implement a get_url(self) which return a
# download URL. The URL returned by this function is passed to curl to
# download the requested model.
class HFDownload:
  def __init__(self, repo_id, filename, subfolder=None, repo_type=None, revision=None):
    self.repo_id = repo_id
    self.filename = filename
    self.subfolder = subfolder
    self.repo_type = repo_type
    self.revision = revision
  
  def get_url(self):
    return hf_url(repo_id=self.repo_id,
                  filename=self.filename,
                  subfolder=self.subfolder,
                  repo_type=self.repo_type,
                  revision=self.revision)

class CurlDownload:
  def __init__(self, url):
    self.url = url
  
  def get_url(self):
    return self.url

# define model download options
AVAILABLE_MODELS = {
    "AbyssOrangeMix2 SFW": HFDownload("WarriorMama777/OrangeMixs",
                                      "AbyssOrangeMix2_sfw.safetensors",
                                      subfolder="Models/AbyssOrangeMix2"),
    "Ally's Mix": CurlDownload("https://civitai.com/api/download/models/1259"),
    "UnstableInkDreamV6": HFDownload("Oosayam/UnstableSamInkDream",
                                     "UnstableinkdreamV6.safetensors"),
}

# TODO: can this be populated automatically?
CHOSEN_MODEL = "AbyssOrangeMix2 SFW" #@param ["AbyssOrangeMix2 SFW", "Ally's Mix", "UnstableInkDreamV6"]

# download the model
DOWNLOAD_URL = AVAILABLE_MODELS[CHOSEN_MODEL].get_url()
%cd models/Stable-diffusion
!curl -LO --remote-name --remote-header-name "$DOWNLOAD_URL"

# start web ui
%cd /content/stable-diffusion-webui
!python launch.py --share